<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B12%5DAgent_Hook_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** LifeCycles **  
There are two types of loops. Agent Loops and Run Loops.


**Agent lifecycle hook**:  Whenever a user ask a query to agent. This query will send to LLM from agent and LLM process this query and check whether it answer this query by itself or call a tool.

**Run Lifecycle**:  If LLM call a tool then its response send to LLM and then LLM send final response to user. You can show these whole process to user. For this we use lifecycle hook.




In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 7.5 MB/s eta 0:00:00


# **Agent Lifecycle**

**On_start** : this event/hook fire before activation of agent.


**On_end** : this event/hook fire at the end of agent working or when agent get final_output.


**On_llm_start** : this hook start when llm processing start.


**On_llm_end** : this hook start when llm processing end.


**On_tool_start**:  before start  execution of tool.


**On_tool_end**: after end of execution of tool.


**On_handoff** : when handoffs


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool,AgentHooks,RunContextWrapper
from google.colab import userdata
from pydantic import BaseModel
api_key = userdata.get('GOOGLE_API_KEY')
base_url = userdata.get('BASE_URL')

client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)
#Pass this class to agent hooks
class weather_hooks(AgentHooks):
  def __init__(self,name):
    self.name=name

  async def on_start(self, context: RunContextWrapper, agent:Agent) -> None:
     print(f'Agent {agent.name} started')

  async def on_llm_start(self, context: RunContextWrapper, agent, system_prompt, input_items) -> None:
     # Access the 'text' field of the first item in the input_items list
     print(f'LLM Start working on {input_items} started')
  async def on_llm_end(self, context: RunContextWrapper, agent, output) -> None:
     print(f'Agent {output} ended')
  async def on_tool_start(self, context: RunContextWrapper, agent, tool) -> None:
     print(f'Agent {tool.name} started')
  async def on_tool_end(self, context: RunContextWrapper, agent, tool, output) -> None:
     print(f'Tool Response :  {output} ended')
  async def on_end(self, context: RunContextWrapper, agent, output) -> None:
     print(f'Agent output :  {output}')
@function_tool()
async def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    return f'The weather for {location} is sunny and temperature is 25C'
set_tracing_disabled(disabled=True)
weather_agent = Agent(
    name='weather_agent',
    instructions='You are helpful agent',
    model=model,
    tools=[get_current_weather],
    hooks=weather_hooks('weather_agent')
    )
response = Runner.run_sync(weather_agent,'How the weather in Multan')
print(response.final_output)

Agent weather_agent started
LLM Start working on [{'content': 'How the weather in Multan', 'role': 'user'}] started
Agent ModelResponse(output=[ResponseFunctionToolCall(arguments='{"location":"Multan"}', call_id='', name='get_current_weather', type='function_call', id='__fake_id__', status=None)], usage=Usage(requests=1, input_tokens=54, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=18, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=135), response_id=None) ended
Agent get_current_weather started
Tool Response :  The weather for Multan is sunny and temperature is 25C ended
LLM Start working on [{'content': 'How the weather in Multan', 'role': 'user'}, {'arguments': '{"location":"Multan"}', 'call_id': '', 'name': 'get_current_weather', 'type': 'function_call', 'id': '__fake_id__'}, {'call_id': '', 'output': 'The weather for Multan is sunny and temperature is 25C', 'type': 'function_call_output'}] started
Agent ModelResponse(output=[R

# **Run Lifecycle**

Run lifecycle: Agent hooks only work for a particular events but run lifecyle would works for whole of the loop.


**On_start_agent** : any agent become active.


**On_agent_end** : any agent end.


**On_llm_start** : when llm start.


**On_llm_end** : when llm end.


**On_tool_start**: when any tool start.


**On_tool_end** : when any tool end.


**On_hand0ff** : when any agents handoff.


For this we run hooks on runner:


# **Chat-Bots**

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool,AgentHooks,RunContextWrapper
from google.colab import userdata
from pydantic import BaseModel
api_key = userdata.get('GOOGLE_API_KEY')
base_url = userdata.get('BASE_URL')

client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)

@function_tool()
async def get_current_weathers(location: str) -> str:
    """Get the current weather in a given location"""
    return f'The weather for {location} is sunny and temperature is 25C'

set_tracing_disabled(disabled=True)
weather_agent = Agent(name='weather_agent', instructions='You are helpful agent', model=model,tools=[get_current_weathers])
user_chat=[]
while True:
  user_input = input("Enter your message: ")
  if user_input.lower()=='exit':
    break
  user_chat.append({'role':'user','content':user_input})
  response = Runner.run_sync(weather_agent,user_chat)
  print(response.final_output)




Enter your message: exit


**# Session**

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool,AgentHooks,RunContextWrapper, SQLiteSession
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
base_url = userdata.get('BASE_URL')

client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)

session  = SQLiteSession('user123', 'user.db')

set_tracing_disabled(disabled=True)
weather_agent = Agent(name='weather_agent', instructions='You are helpful agent', model=model)
response = Runner.run_sync(weather_agent,'what is my name', session=session)
print(response.final_output)




I don't know your name. As an AI, I don't have access to your personal information.

You'd have to tell me! What would you like me to call you?


# **Memory**

In [ ]:
pip install -Uq mem0ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 11.4 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled,function_tool
from google.colab import userdata
from mem0 import MemoryClient

api_key = userdata.get('GOOGLE_API_KEY')
base_url = userdata.get('BASE_URL')
mem0_api_key = userdata.get('MEM0')

mem_client = MemoryClient(api_key=mem0_api_key)


client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)

@function_tool()
async def get_Search_memory(query:str):
  """Search memory"""
  print('Searching query')
  return  mem_client.search(query=query,user_id = 'poo1',top_k=3)
@function_tool()
async def saving_memory(query:str):
  """Saving memory"""
  print('Saving memory')
  return mem_client.add([{'role':'user', 'content':query}],user_id = 'poo1')

set_tracing_disabled(disabled=True)
weather_agent = Agent(name='weather_agent', instructions='You are helpful agent use tool get_search_memory for getting info', model=model,tools=[get_Search_memory,saving_memory])
response = Runner.run_sync(weather_agent,'What my name')
print(response.final_output)

Searching query
Your name is Ali.
